<a href="https://colab.research.google.com/github/imbalzy/RecipeQA-FInal-Project-2470/blob/main/visual_ordering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!cp "/content/gdrive/My Drive/Kaggle/kaggle.json" .

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!kaggle datasets download -d jeromeblanchet/recipeqa-nlp-dataset
!unzip recipeqa-nlp-dataset.zip -d data/ > /dev/null

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

recipeqa-nlp-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
replace data/images/images-qa/test/images-qa/-homebrew-honey-beer-_1_2.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: --2020-12-09 19:59:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-09 19:59:39--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-09 19:59:39--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
C

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from preprocessrecipeqa import *
import random

In [4]:
batch_size = 16
(train_iter1, train_iter2, train_iter3, train_iter4), (test_iter1, test_iter2, test_iter3, test_iter4), (val_iter1, val_iter2, val_iter3, val_iter4), embedding_index, word_index = preprocess(batch_size)
l_embed = get_embedding_layer(word_index, embedding_index)


In [5]:
def define_model():
  conv = keras.Sequential()
  conv.add(keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same'))
  conv.add(keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same'))
  conv.add(keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same'))
  conv.add(keras.layers.Conv2D(filters = 64, kernel_size = 5, strides=(2, 2), padding='same'))
  conv.add(keras.layers.Flatten())
  conv.add(keras.layers.Dense(500))
  inputs1 = keras.Input(shape=(224, 224, 3))
  inputs2 = keras.Input(shape=(224, 224, 3))
  inputs3 = keras.Input(shape=(224, 224, 3))
  inputs4 = keras.Input(shape=(224, 224, 3))
  concat = keras.layers.Concatenate()
  x = concat([conv(inputs1),conv(inputs2),conv(inputs3),conv(inputs4)])
  fc1 = keras.layers.Dense(500,activation = 'relu')
  fc2 = keras.layers.Dense(500,activation = 'relu')
  fc3 = keras.layers.Dense(500,activation = 'relu')
  fc4 = keras.layers.Dense(2,activation = 'softmax')
  outputs = fc4(fc3(fc2(fc1(x))))
  model = tf.keras.Model(inputs=[inputs1,inputs2,inputs3,inputs4], outputs=outputs)
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
  return model
model =define_model()

In [6]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
_______________________________________________________________________________________

In [7]:
labels=[1 if j%2==0 else 0 for j in range(batch_size*2)]
labels=tf.convert_to_tensor(labels)
for i, label in train_iter4:
  inputs1=[]
  inputs2=[]
  inputs3=[]
  inputs4=[]
  for j in range(batch_size):
    # print(i[0]['recipe_id'])
    comb = [i[j]['image_list'][k] for k in i[j]['choice_list'][label[j]]]
    inputs1.append(comb[0])
    inputs2.append(comb[1])
    inputs3.append(comb[2])
    inputs4.append(comb[3])
    if label[j]==0:
      l=1
    else:
      l=0
    comb = [i[j]['image_list'][k] for k in i[j]['choice_list'][l]]
    inputs1.append(comb[0])
    inputs2.append(comb[1])
    inputs3.append(comb[2])
    inputs4.append(comb[3])
  inputs1 = tf.convert_to_tensor(inputs1)
  inputs2 = tf.convert_to_tensor(inputs2)
  inputs3 = tf.convert_to_tensor(inputs3)
  inputs4 = tf.convert_to_tensor(inputs4)
  print(model.train_on_batch([inputs1,inputs2,inputs3,inputs4],labels))
  

[0.710526704788208, 0.5]
[0.7005174160003662, 0.5625]
[1.5322054624557495, 0.4375]
[1.9218182563781738, 0.53125]
[1.1580244302749634, 0.5]
[1.125509262084961, 0.65625]
[0.9253045320510864, 0.59375]
[0.8254854679107666, 0.40625]
[0.7173489332199097, 0.4375]
[0.740043044090271, 0.5625]
[1.0847491025924683, 0.5]
[1.5643231868743896, 0.5]
[0.7757695913314819, 0.59375]
[0.9103328585624695, 0.5]
[1.044370174407959, 0.46875]
[1.0300631523132324, 0.53125]
[0.727121114730835, 0.375]
[0.915818452835083, 0.5]
[0.7378081679344177, 0.5625]
[0.8114617466926575, 0.4375]
[1.0210657119750977, 0.46875]
[2.295893907546997, 0.53125]
[1.575046181678772, 0.46875]
[0.7392062544822693, 0.59375]
[0.861152172088623, 0.46875]
[3.7279250621795654, 0.46875]
[0.780994713306427, 0.40625]
[0.6838462352752686, 0.5625]
[0.7531671524047852, 0.4375]
[0.8523226976394653, 0.5625]
[0.6637786626815796, 0.65625]
[0.699500322341919, 0.4375]
[0.8307613730430603, 0.375]
[0.8713847994804382, 0.46875]
[0.738632082939148, 0.5]
[0.7

KeyboardInterrupt: ignored

In [14]:
all=[]
for i, label in val_iter4:
  inputs1=[]
  inputs2=[]
  inputs3=[]
  inputs4=[]
  # print(label)
  for j in range(batch_size):
    # print(i[0]['recipe_id'])

    for l in range(4):
      comb = [i[j]['image_list'][k] for k in i[j]['choice_list'][l]]
      inputs1.append(comb[0])
      inputs2.append(comb[1])
      inputs3.append(comb[2])
      inputs4.append(comb[3])
  inputs1 = tf.convert_to_tensor(inputs1)
  inputs2 = tf.convert_to_tensor(inputs2)
  inputs3 = tf.convert_to_tensor(inputs3)
  inputs4 = tf.convert_to_tensor(inputs4)
  answer = model.predict([inputs1,inputs2,inputs3,inputs4])
  array = []
  for k in range(0,len(answer),4):
    m = answer[k][1]
    index=0
    for l in range(1,4):
      if m<answer[k+l][1]:
        m=answer[k+l][1]
        index=l
    array.append(index)
  correct = 0
  for j in range(batch_size):
    if label[j]==array[j]:
      correct+=1
  all.append(correct/batch_size)
  print(sum(all)/len(all))
  

0.1875
0.1875
0.20833333333333334
0.21875
0.25
0.2708333333333333
0.25892857142857145
0.2578125
0.2638888888888889
0.26875
0.26704545454545453
0.2916666666666667
0.27884615384615385
0.28125
0.2916666666666667
0.29296875
0.29411764705882354
0.2951388888888889
0.29605263157894735
0.303125
0.3125
0.3153409090909091
0.3179347826086957
0.3255208333333333
0.3225
0.31971153846153844
0.3125
0.3080357142857143
0.3146551724137931
0.31666666666666665
0.3125
0.31640625
0.3143939393939394
0.3088235294117647
0.3107142857142857
0.3055555555555556
0.30405405405405406
0.3026315789473684
0.30448717948717946
0.30625
0.31097560975609756
0.30952380952380953
0.3066860465116279
0.3039772727272727
0.3
0.3016304347826087
0.2978723404255319
0.29296875
0.29336734693877553
0.29375
0.29044117647058826
0.2956730769230769
0.29952830188679247
0.2986111111111111
0.29545454545454547
0.29575892857142855
0.2982456140350877
0.2984913793103448
